# Arboles de Expansion

Tanto el algoritmo de Kruskal como el de Prim son algoritmos voraces utilizados para encontrar un Árbol de Expansión Mínima (MST) en un grafo conexo y ponderado. Un MST es un subconjunto de aristas que conecta todos los vértices del grafo sin crear ciclos, y la suma de los pesos de las aristas en el AEM es la mínima posible entre todos los MST del grafo.

In [ ]:
class abstract_graph:
    
    def __init__(self,_edges):
        self.edges=_edges
        self.nodes={u for u,v in self.edges} | {v for u,v in self.edges}
        
    def adjacency_matrix(self):
        pass
    
    def adjacency_list(self):
        pass

    
import numpy as np

class simple_graph(abstract_graph):
    
    def __init__(self,_edges):
        tmp=[]
        for (u,v) in _edges:
            tmp.append((u,v))
            if (v,u) not in tmp and v!=u:
                tmp.append((v,u))
        self.edges=tmp
        self.nodes={u for u,v in _edges} | {v for u,v in _edges}
     
    def adjacency_matrix(self):
        # completar
        n=len(self.nodes)
        mat=np.zeros((n,n))
        for i,v in enumerate(self.nodes):
            for j,k in enumerate(self.nodes):
                if (v,k) in self.edges:
                    mat[i,j]=1
        return mat
    
    
    def adjacency_list(self):
        adjacent=lambda n : {v for u,v in self.edges if u==n } 
        return {v:adjacent(v) for v in self.nodes}

  
class weighted_graph(simple_graph):
    
    def __init__(self,_edges):
        tmp=dict()
        for (u,v),w in _edges.items():
            tmp.update({(u,v):w})
            if (v,u) not in tmp.keys() and v!=u:
                tmp.update({(v,u):w})
        self.edges=tmp
        self.nodes={u for u,v in _edges} | {v for u,v in _edges}

    def adjacency_matrix(self):
        # completar
        n=len(self.nodes)
        mat=np.zeros((n,n))
        for i,v in enumerate(self.nodes):
            for j,k in enumerate(self.nodes):
                if (v,k) in self.edges:
                    mat[i,j]=self.edges[(v,k)]
        return mat

# Algoritmo Kruskal


1. Ordenar las aristas: Se ordenan todas las aristas del grafo de menor a mayor peso.

2. Inicializar un bosque: Se crea un bosque $C$, donde cada vértice del grafo es un árbol separado.

3. Recorrer las aristas ordenadas: Se recorren las aristas ordenadas por peso. 

4. Para cada arista:
    1. Si la arista no crea un ciclo en el bosque $C$:
        - Se agrega la arista al bosque $C$.
        - Se unen los dos árboles que contenían los extremos de la arista en un solo árbol.
    2. Si la arista crea un ciclo en el bosque $C$, se ignora.

Retorno: El bosque $C$ final es el MST del grafo.

In [99]:
def naive_find(C, u):
    while C[u] != u:
        u = C[u]
    return u

def naive_union(C, u, v):
    u = naive_find(C, u)
    v = naive_find(C, v)
    C[u]=v
        
def kruskal(G):
    E = G.edges
    T=[]
    C= {u:u for u in G.nodes}
    for (u,v),weight in sorted(E.items(), key=lambda weighted_edge: weighted_edge[1]):
        if naive_find(C, u) != naive_find(C, v): 
            T.append((u,v,{'weight':weight}))
            naive_union(C, u, v)
    return T

In [68]:
E={(1,2):4,(2,3):7,(3,4):8,(3,5):10,(4,5):9}
G=weighted_graph(E)
print('aristas : ',G.edges)

aristas :  {(1, 2): 4, (2, 1): 4, (2, 3): 7, (3, 2): 7, (3, 4): 8, (4, 3): 8, (3, 5): 10, (5, 3): 10, (4, 5): 9, (5, 4): 9}


In [69]:
kruskal(G)

[(1, 2, {'weight': 4}),
 (2, 3, {'weight': 7}),
 (3, 4, {'weight': 8}),
 (4, 5, {'weight': 9})]

In [72]:
import networkx as nx

G_nx=nx.Graph()
G_nx.add_weighted_edges_from([(u,v,w) for ((u,v),w) in E.items()])
T_nx=nx.minimum_spanning_tree(G_nx,algorithm='kruskal')

In [73]:
T_nx.edges(data=True)

EdgeDataView([(1, 2, {'weight': 4}), (2, 3, {'weight': 7}), (3, 4, {'weight': 8}), (4, 5, {'weight': 9})])

# Algoritmo Prim


1. Elegir un vértice inicial: Se elige un vértice arbitrario del grafo como vértice inicial.

2. Inicializar un conjunto $V$: El conjunto $V$ inicialmente contiene solo el vértice inicial.

3. Recorrer los vértices no incluidos en $V$: 

    1. Se recorren los vértices del grafo que no están incluidos en $V$. 
    2. Para cada vértice v:
        - Se encuentra la arista de menor peso que conecta un vértice de $V$ con $v$.
        - Si la arista existe:
            * Se agrega la arista a $V$.
            * Se agrega el vértice $v$ a $V$.

4. Retorno: El grafo inducido por los vértices y aristas de $V$ es el MST del grafo.

In [74]:
from heapq import heappop, heappush

def prim(G, s):
    V, Q = {}, [(0, None, s)]
    L=G.adjacency_list()
    T=list()
    while Q:
        w, p, u = heappop(Q)
        if u in V: continue
        V[u] = p
        T.append((p,u,{'weight':w}))
        for v in L[u]:
            weight=G.edges[(u,v)]
            heappush(Q, (weight, u, v))
    return T[1:]


In [76]:
prim(G,1)

[(1, 2, {'weight': 4}),
 (2, 3, {'weight': 7}),
 (3, 4, {'weight': 8}),
 (4, 5, {'weight': 9})]

In [77]:
T_nx=nx.minimum_spanning_tree(G_nx,algorithm='prim')
T_nx.edges(data=True)

EdgeDataView([(1, 2, {'weight': 4}), (2, 3, {'weight': 7}), (3, 4, {'weight': 8}), (4, 5, {'weight': 9})])

# MST en Grafos Aleatorios

In [78]:
import networkx as nx
import numpy as np 

def gen_random_graph(n,p):
    not_connected=True
    while not_connected:
        G_nx = nx.erdos_renyi_graph(int(n),p,directed=False)
        not_connected=nx.is_connected(G_nx)
        weights={edge:np.random.randint(1,10) for edge in G_nx.edges}
        nx.set_edge_attributes(G_nx, values = weights, name = 'weight')
        break
    return G_nx

In [79]:
G_nx=gen_random_graph(1e2,0.2)

In [80]:
E={(u,v):k['weight'] for (u,v,k) in G_nx.edges(data=True)}
G=weighted_graph(E)

In [81]:
import time 

t1=time.time()
T_nx_kruskal=nx.minimum_spanning_tree(G_nx,algorithm='kruskal')
t2=time.time()
print('Algoritmo Kruskal NX Tiempo : {0:2f}[s], Peso : {1}'.format(t2-t1,sum([k['weight'] for (u,v,k) in T_nx_kruskal.edges(data=True)])))

Algoritmo Kruskal NX Tiempo : 0.002881[s], Peso : 109


In [82]:
t1=time.time()
T_jax_kruskal=kruskal(G)
t2=time.time()
print('Algoritmo Kruskal Tiempo : {0:2f}[s], Peso : {1}'.format(t2-t1,sum([k['weight'] for (u,v,k) in T_jax_kruskal])))

Algoritmo Kruskal Tiempo : 0.005255[s], Peso : 109


In [83]:
t1=time.time()
T_jax_prim=prim(G,0)
t2=time.time()
print('Algoritmo Prim Tiempo : {0:2f}[s], Peso : {1}'.format(t2-t1,sum([k['weight'] for (u,v,k) in T_jax_prim])))

Algoritmo Prim Tiempo : 0.010802[s], Peso : 109


In [84]:
import scipy as sp
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.sparse import csr_matrix

M = nx.adjacency_matrix(G_nx)
t1=time.time()
T_sp=minimum_spanning_tree(csr_matrix(M))
t2=time.time()
print('Algoritmo Kruskal SciPy Tiempo : {0:2f}[s], Peso : {1}'.format(t2-t1,np.sum(T_sp)))

Algoritmo Kruskal SciPy Tiempo : 0.001608[s], Peso : 109.0


# Cython MST

In [92]:
import heapq 
import numpy as np 
import priority_queue as pq

def test_heap_insert(randlist):
    heap=list()
    for item in randlist:
        heapq.heappush(heap, (item, item))
    for _ in range(len(randlist)):
        heapq.heappop(heap)

def test_pq_insert(randlist):
    Q=pq.PyPQ()
    for item in randlist:
       Q.insert((item, item))
    for _ in range(len(randlist)):
        Q.pop()


In [93]:
randlist=np.random.permutation(range(10_000_000))

In [95]:
%%prun -s cumulative -q -l 10 -T prun0
import time 

t1=time.time()
test_pq(randlist)
t2=time.time()
print(t2-t1)

3.9868550300598145
 
*** Profile printout saved to text file 'prun0'.


In [96]:
print(open('prun0', 'r').read())

         31 function calls in 3.988 seconds

   Ordered by: cumulative time
   List reduced from 22 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.988    3.988 {built-in method builtins.exec}
        1    0.123    0.123    3.988    3.988 <string>:1(<module>)
        1    3.863    3.863    3.863    3.863 397704518.py:12(test_pq)
        1    0.000    0.000    0.001    0.001 {built-in method builtins.print}
        2    0.000    0.000    0.000    0.000 iostream.py:610(write)
        2    0.000    0.000    0.000    0.000 iostream.py:532(_schedule_flush)
        1    0.000    0.000    0.000    0.000 iostream.py:243(schedule)
        1    0.000    0.000    0.000    0.000 socket.py:621(send)
        1    0.000    0.000    0.000    0.000 threading.py:1185(is_alive)
        2    0.000    0.000    0.000    0.000 iostream.py:505(_is_master_process)


In [97]:
%%prun -s cumulative -q -l 10 -T prun1
t1=time.time()
test_heap(randlist)
t2=time.time()
print(t2-t1)

3.0764729976654053
 
*** Profile printout saved to text file 'prun1'.


In [98]:
print(open('prun1', 'r').read())

         10001031 function calls in 3.077 seconds

   Ordered by: cumulative time
   List reduced from 24 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.077    3.077 {built-in method builtins.exec}
        1    0.335    0.335    3.077    3.077 <string>:1(<module>)
        1    1.811    1.811    2.742    2.742 397704518.py:5(test_heap)
 10000000    0.915    0.000    0.915    0.000 {built-in method _heapq.heappush}
     1000    0.016    0.000    0.016    0.000 {built-in method _heapq.heappop}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        2    0.000    0.000    0.000    0.000 iostream.py:610(write)
        2    0.000    0.000    0.000    0.000 iostream.py:532(_schedule_flush)
        1    0.000    0.000    0.000    0.000 iostream.py:243(schedule)
        1    0.000    0.000    0.000    0.000 socket.py:621(send)
